# Módulo 9: Foundation Models para Visão Computacional## Objetivos de Aprendizagem- Compreender o conceito de Foundation Models- Conhecer modelos como CLIP, DALL-E e GPT-4V- Implementar soluções com APIs do OpenAI e Gemini- Analisar vantagens e limitações dos Foundation Models- Aplicar Foundation Models em casos práticos do mercado---## 9.1 Introdução aos Foundation Models**Foundation Models** são modelos de inteligência artificial treinados em grandes quantidades de dados não rotulados que podem ser adaptados para uma ampla gama de tarefas downstream através de técnicas como fine-tuning, prompt engineering e few-shot learning.![Introdução Foundation Models](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo9/introducao_foundation_models.png)### Conceitos Fundamentais**Definição de Foundation Models:**- **Treinamento em escala**: Modelos treinados em datasets massivos- **Capacidade de adaptação**: Adaptação para múltiplas tarefas- **Emergência de capacidades**: Habilidades que emergem com escala- **Base para aplicações**: Fundação para sistemas especializados**Características Principais:**- **Escala**: Milhões ou bilhões de parâmetros- **Dados**: Treinamento em datasets diversificados- **Multimodalidade**: Capacidade de processar diferentes modalidades- **Generalização**: Performance em tarefas não vistas durante treinamento### Evolução dos Foundation Models**Cronologia dos Marcos:**- **2017**: Transformer architecture (Attention Is All You Need)- **2018**: BERT (Bidirectional Encoder Representations)- **2019**: GPT-2 (Generative Pre-trained Transformer)- **2020**: GPT-3 (175B parameters)- **2021**: CLIP (Contrastive Language-Image Pre-training)- **2022**: DALL-E 2, ChatGPT, Stable Diffusion- **2023**: GPT-4, GPT-4V, Gemini, Claude![Evolução Foundation Models](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo9/evolucao_foundation_models.png)**Marcos Importantes:**- **2017**: Attention Is All You Need - Vaswani et al.- **2018**: BERT: Pre-training of Deep Bidirectional Transformers - Devlin et al.- **2020**: Language Models are Few-Shot Learners - Brown et al.- **2021**: CLIP: Learning Transferable Visual Representations - Radford et al.- **2022**: DALL-E 2: Hierarchical Text-Conditional Image Generation - Ramesh et al.**Referências:**- [On the Opportunities and Risks of Foundation Models - Bommasani et al.](https://arxiv.org/abs/2108.07258)- [CLIP: Learning Transferable Visual Representations - Radford et al.](https://arxiv.org/abs/2103.00020)

## 9.2 Demonstração Prática: Foundation Models Simples

Vamos implementar e demonstrar conceitos de Foundation Models:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import json
from typing import List, Dict, Tuple

class FoundationModelsDemo:
    """Demonstração de conceitos de Foundation Models"""
    
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def create_synthetic_dataset(self, num_samples=1000):
        """Cria dataset sintético para demonstração"""
        
        # Simular dados de texto e imagem
        text_data = []
        image_data = []
        labels = []
        
        categories = [
            {"text": "a red car", "image": [1, 0, 0], "label": 0},
            {"text": "a blue car", "image": [0, 1, 0], "label": 1},
            {"text": "a green car", "image": [0, 0, 1], "label": 2},
            {"text": "a red truck", "image": [1, 0, 0], "label": 3},
            {"text": "a blue truck", "image": [0, 1, 0], "label": 4},
            {"text": "a green truck", "image": [0, 0, 1], "label": 5}
        ]
        
        for i in range(num_samples):
            # Selecionar categoria aleatória
            category = categories[i % len(categories)]
            
            # Adicionar variação ao texto
            text_variations = [
                category["text"],
                f"{category['text']} in the street",
                f"{category['text']} parked outside",
                f"{category['text']} driving fast",
                f"{category['text']} at the traffic light"
            ]
            
            text = text_variations[i % len(text_variations)]
            
            # Adicionar ruído à imagem
            image = np.array(category["image"]) + np.random.normal(0, 0.1, 3)
            image = np.clip(image, 0, 1)
            
            text_data.append(text)
            image_data.append(image)
            labels.append(category["label"])
        
        return text_data, image_data, labels
    
    def simple_text_encoder(self, text_data, vocab_size=1000, embedding_dim=128):
        """Simula um encoder de texto simples"""
        
        # Criar vocabulário simples
        vocab = {}
        for text in text_data:
            words = text.lower().split()
            for word in words:
                if word not in vocab:
                    vocab[word] = len(vocab)
        
        # Embedding layer
        embedding = nn.Embedding(len(vocab), embedding_dim)
        
        # Encoder simples
        encoder = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )
        
        # Processar textos
        text_embeddings = []
        for text in text_data:
            words = text.lower().split()
            word_ids = [vocab.get(word, 0) for word in words]
            
            if word_ids:
                # Média dos embeddings das palavras
                word_embeddings = embedding(torch.tensor(word_ids))
                text_embedding = torch.mean(word_embeddings, dim=0)
            else:
                text_embedding = torch.zeros(embedding_dim)
            
            text_embeddings.append(text_embedding)
        
        # Aplicar encoder
        text_embeddings = torch.stack(text_embeddings)
        encoded_texts = encoder(text_embeddings)
        
        return encoded_texts, vocab
    
    def simple_image_encoder(self, image_data, embedding_dim=128):
        """Simula um encoder de imagem simples"""
        
        # Encoder simples para imagens
        encoder = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )
        
        # Processar imagens
        image_tensor = torch.FloatTensor(image_data)
        encoded_images = encoder(image_tensor)
        
        return encoded_images
    
    def contrastive_learning(self, text_embeddings, image_embeddings, temperature=0.07):
        """Simula aprendizado contrastivo"""
        
        # Normalizar embeddings
        text_embeddings = F.normalize(text_embeddings, p=2, dim=1)
        image_embeddings = F.normalize(image_embeddings, p=2, dim=1)
        
        # Calcular similaridade
        similarity = torch.matmul(text_embeddings, image_embeddings.T) / temperature
        
        # Labels para contraste (diagonal = positivos)
        batch_size = text_embeddings.size(0)
        labels = torch.arange(batch_size).to(text_embeddings.device)
        
        # Loss contrastivo
        loss_text = F.cross_entropy(similarity, labels)
        loss_image = F.cross_entropy(similarity.T, labels)
        
        total_loss = (loss_text + loss_image) / 2
        
        return total_loss, similarity
    
    def zero_shot_classification(self, text_embeddings, image_embeddings, test_texts, test_images):
        """Demonstra classificação zero-shot"""
        
        # Normalizar embeddings
        text_embeddings = F.normalize(text_embeddings, p=2, dim=1)
        image_embeddings = F.normalize(image_embeddings, p=2, dim=1)
        
        # Calcular similaridade
        similarity = torch.matmul(image_embeddings, text_embeddings.T)
        
        # Predições
        predictions = torch.argmax(similarity, dim=1)
        
        return predictions, similarity
    
    def visualize_embeddings(self, text_embeddings, image_embeddings, labels, title="Embeddings"):
        """Visualiza os embeddings em 2D"""
        
        # Reduzir dimensionalidade usando PCA simples
        from sklearn.decomposition import PCA
        
        # Combinar embeddings
        all_embeddings = torch.cat([text_embeddings, image_embeddings], dim=0)
        
        # PCA para 2D
        pca = PCA(n_components=2)
        embeddings_2d = pca.fit_transform(all_embeddings.detach().numpy())
        
        # Separar texto e imagem
        text_2d = embeddings_2d[:len(text_embeddings)]
        image_2d = embeddings_2d[len(text_embeddings):]
        
        # Visualizar
        plt.figure(figsize=(12, 8))
        
        # Cores para diferentes labels
        colors = plt.cm.Set3(np.linspace(0, 1, len(set(labels))))
        
        for i, label in enumerate(set(labels)):
            mask = np.array(labels) == label
            
            # Texto
            plt.scatter(text_2d[mask, 0], text_2d[mask, 1], 
                       c=[colors[label]], marker='o', s=100, 
                       label=f'Text - Label {label}', alpha=0.7)
            
            # Imagem
            plt.scatter(image_2d[mask, 0], image_2d[mask, 1], 
                       c=[colors[label]], marker='s', s=100, 
                       label=f'Image - Label {label}', alpha=0.7)
        
        plt.title(f'{title} - Text (circles) vs Image (squares)')
        plt.xlabel('PC1')
        plt.ylabel('PC2')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        return embeddings_2d
    
    def analyze_performance(self, predictions, true_labels, similarity_matrix):
        """Analisa a performance do modelo"""
        
        # Calcular accuracy
        accuracy = (predictions == true_labels).float().mean().item()
        
        # Análise de similaridade
        similarity_scores = torch.diag(similarity_matrix)
        
        # Visualizar resultados
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Histograma de accuracy
        axes[0].hist(similarity_scores.detach().numpy(), bins=20, alpha=0.7)
        axes[0].set_title(f'Distribuição de Similaridade\nAccuracy: {accuracy:.3f}')
        axes[0].set_xlabel('Similaridade')
        axes[0].set_ylabel('Frequência')
        axes[0].grid(True, alpha=0.3)
        
        # Matriz de confusão
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(true_labels, predictions)
        
        im = axes[1].imshow(cm, interpolation='nearest', cmap='Blues')
        axes[1].set_title('Matriz de Confusão')
        axes[1].set_xlabel('Predição')
        axes[1].set_ylabel('Verdadeiro')
        
        # Adicionar valores na matriz
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                axes[1].text(j, i, cm[i, j], ha='center', va='center')
        
        plt.colorbar(im, ax=axes[1])
        plt.tight_layout()
        plt.show()
        
        return {
            'accuracy': accuracy,
            'similarity_scores': similarity_scores,
            'confusion_matrix': cm
        }

def demonstrate_foundation_models():
    """Demonstra conceitos de Foundation Models"""
    
    print("=== Demonstração de Foundation Models ===")
    print("\nNota: Esta demonstração usa dados sintéticos para fins educacionais.")
    print("Em aplicações reais, use datasets reais e modelos pré-treinados.")
    
    # Criar instância
    demo = FoundationModelsDemo()
    
    # Criar dataset sintético
    print("\nCriando dataset sintético...")
    text_data, image_data, labels = demo.create_synthetic_dataset(num_samples=500)
    
    # Encoder de texto
    print("\nProcessando texto...")
    text_embeddings, vocab = demo.simple_text_encoder(text_data)
    
    # Encoder de imagem
    print("\nProcessando imagens...")
    image_embeddings = demo.simple_image_encoder(image_data)
    
    # Aprendizado contrastivo
    print("\nExecutando aprendizado contrastivo...")
    loss, similarity = demo.contrastive_learning(text_embeddings, image_embeddings)
    print(f"Loss contrastivo: {loss.item():.4f}")
    
    # Visualizar embeddings
    print("\nVisualizando embeddings...")
    embeddings_2d = demo.visualize_embeddings(text_embeddings, image_embeddings, labels)
    
    # Classificação zero-shot
    print("\nExecutando classificação zero-shot...")
    test_texts = text_data[:100]
    test_images = image_data[:100]
    test_labels = labels[:100]
    
    predictions, similarity_matrix = demo.zero_shot_classification(
        text_embeddings[:100], image_embeddings[:100], test_texts, test_images
    )
    
    # Analisar performance
    print("\nAnalisando performance...")
    performance = demo.analyze_performance(predictions, test_labels, similarity_matrix)
    
    return demo, text_embeddings, image_embeddings, performance

# Executar demonstração
demo_model, text_emb, image_emb, model_performance = demonstrate_foundation_models()

### Análise dos Resultados

**Foundation Models Observados:**

1. **Embeddings**: Representações aprendidas de texto e imagem
2. **Aprendizado Contrastivo**: Alinhamento entre modalidades
3. **Classificação Zero-Shot**: Predição sem treinamento específico
4. **Performance**: Accuracy e similaridade

**Insights Importantes:**
- **Alinhamento**: Texto e imagem no mesmo espaço
- **Generalização**: Funciona em dados não vistos
- **Escalabilidade**: Performance melhora com escala
- **Flexibilidade**: Adaptável a diferentes tarefas

**Referências:**
- [CLIP: Learning Transferable Visual Representations - Radford et al.](https://arxiv.org/abs/2103.00020)


## 9.3 CLIP: Contrastive Language-Image Pre-training**CLIP** é um modelo que aprende representações visuais através de aprendizado contrastivo em pares de texto e imagem, permitindo classificação zero-shot e busca multimodal.![CLIP Model](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo9/clip_model.png)### Arquitetura CLIP**Componentes Principais:****1. Text Encoder:**- **Transformer**: Arquitetura baseada em attention- **Vocabulário**: Vocabulário expandido- **Embedding**: Representações de texto- **Normalização**: Normalização L2**2. Image Encoder:**- **Vision Transformer**: ViT ou ResNet- **Patches**: Divisão em patches- **Embedding**: Representações de imagem- **Normalização**: Normalização L2![Arquitetura CLIP](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo9/arquitetura_clip.png)**3. Contrastive Learning:**- **Similaridade**: Cálculo de similaridade coseno- **Contraste**: Pares positivos e negativos- **Loss**: InfoNCE loss- **Otimização**: Otimização conjunta### Processo de Treinamento**1. Pré-processamento:**- **Texto**: Tokenização e embedding- **Imagem**: Redimensionamento e normalização- **Pares**: Associação texto-imagem- **Batch**: Organização em batches**2. Forward Pass:**- **Text Encoder**: Texto → embedding- **Image Encoder**: Imagem → embedding- **Normalização**: Normalização L2- **Similaridade**: Cálculo de similaridade**3. Loss Function:**- **InfoNCE**: Loss contrastivo- **Temperatura**: Parâmetro de temperatura- **Otimização**: Otimização conjunta- **Convergência**: Alinhamento das modalidades### Aplicações do CLIP**1. Classificação Zero-Shot:**- **Descrições**: Descrições textuais das classes- **Similaridade**: Cálculo de similaridade- **Predição**: Classe com maior similaridade- **Flexibilidade**: Novas classes sem retreinamento**2. Busca Multimodal:**- **Query**: Consulta em texto- **Database**: Base de imagens- **Ranking**: Ordenação por similaridade- **Retrieval**: Recuperação de imagens**3. Transfer Learning:**- **Fine-tuning**: Ajuste fino para tarefas específicas- **Feature Extraction**: Extração de características- **Downstream Tasks**: Tarefas downstream- **Performance**: Melhoria de performance**Referências:**- [CLIP: Learning Transferable Visual Representations - Radford et al.](https://arxiv.org/abs/2103.00020)

## 9.4 DALL-E: Text-to-Image Generation**DALL-E** é um modelo generativo que cria imagens a partir de descrições textuais, combinando técnicas de geração de imagens com processamento de linguagem natural.![DALL-E Model](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo9/dalle_model.png)### Arquitetura DALL-E**Componentes Principais:****1. Text Encoder:**- **Transformer**: Encoder baseado em attention- **Vocabulário**: Vocabulário expandido- **Embedding**: Representações de texto- **Contexto**: Contexto global**2. Image Decoder:**- **Transformer**: Decoder baseado em attention- **Patches**: Divisão em patches- **Geração**: Geração de patches- **Reconstrução**: Reconstrução da imagem![Arquitetura DALL-E](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo9/arquitetura_dalle.png)**3. VQ-VAE:**- **Quantização**: Quantização vetorial- **Codebook**: Livro de códigos- **Compressão**: Compressão de imagem- **Reconstrução**: Reconstrução de alta qualidade### Processo de Geração**1. Pré-processamento:**- **Texto**: Tokenização e embedding- **Imagem**: Divisão em patches- **Quantização**: Quantização vetorial- **Sequência**: Sequência de tokens**2. Geração:**- **Text Encoder**: Texto → contexto- **Image Decoder**: Contexto → patches- **Sampling**: Amostragem autoregressiva- **Reconstrução**: Reconstrução da imagem**3. Pós-processamento:**- **Desquantização**: Desquantização vetorial- **Reconstrução**: Reconstrução da imagem- **Refinamento**: Refinamento da qualidade- **Output**: Imagem final### Aplicações do DALL-E**1. Geração de Imagens:**- **Descrições**: Descrições textuais- **Criação**: Criação de imagens- **Qualidade**: Alta qualidade- **Diversidade**: Grande diversidade**2. Edição de Imagens:**- **Inpainting**: Preenchimento de regiões- **Outpainting**: Extensão da imagem- **Edição**: Edição baseada em texto- **Controle**: Controle preciso**3. Aplicações Criativas:**- **Arte**: Criação artística- **Design**: Design gráfico- **Conteúdo**: Conteúdo visual- **Prototipagem**: Prototipagem rápida![Aplicações DALL-E](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo9/aplicacoes_dalle.png)**Referências:**- [DALL-E: Creating Images from Text - Ramesh et al.](https://arxiv.org/abs/2102.12092)

## 9.5 GPT-4V e Multimodal AI**GPT-4V** é uma versão multimodal do GPT-4 que pode processar tanto texto quanto imagens, permitindo análise visual e geração de conteúdo multimodal.![GPT-4V Model](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo9/gpt4v_model.png)### Arquitetura GPT-4V**Componentes Principais:****1. Vision Encoder:**- **Transformer**: Encoder baseado em attention- **Patches**: Divisão em patches- **Embedding**: Representações de imagem- **Normalização**: Normalização de camadas**2. Text Encoder:**- **Transformer**: Encoder baseado em attention- **Vocabulário**: Vocabulário expandido- **Embedding**: Representações de texto- **Contexto**: Contexto global**3. Multimodal Fusion:**- **Cross-Attention**: Atenção cruzada- **Fusão**: Fusão de modalidades- **Contexto**: Contexto multimodal- **Decodificação**: Decodificação conjunta### Capacidades do GPT-4V**1. Análise Visual:**- **Descrição**: Descrição de imagens- **Análise**: Análise de conteúdo- **Detecção**: Detecção de objetos- **Interpretação**: Interpretação de cenas**2. Geração Multimodal:**- **Texto**: Geração de texto- **Imagem**: Análise de imagem- **Combinação**: Combinação de modalidades- **Criatividade**: Conteúdo criativo**3. Aplicações Práticas:**- **Assistência**: Assistência visual- **Educação**: Educação multimodal- **Acessibilidade**: Acessibilidade visual- **Produtividade**: Produtividade multimodal### Demonstração Prática: APIsVamos demonstrar o uso de APIs para Foundation Models:

In [ ]:
import requests
import json
import base64
from typing import Dict, List, Optional

class FoundationModelsAPI:
    """Demonstração de APIs para Foundation Models"""
    
    def __init__(self):
        self.openai_api_key = None  # Configurar com sua chave
        self.gemini_api_key = None  # Configurar com sua chave
        
    def create_sample_image(self, width=64, height=64):
        """Cria uma imagem de exemplo para demonstração"""
        
        import numpy as np
        import matplotlib.pyplot as plt
        
        # Criar imagem simples
        img = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Adicionar formas
        # Círculo vermelho
        center = (width//2, height//2)
        radius = min(width, height) // 4
        
        y, x = np.ogrid[:height, :width]
        mask = (x - center[0])**2 + (y - center[1])**2 <= radius**2
        img[mask] = [255, 0, 0]  # Vermelho
        
        # Salvar imagem
        plt.imsave('sample_image.png', img)
        
        return img
    
    def encode_image_to_base64(self, image_path: str) -> str:
        """Codifica imagem para base64"""
        
        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        
        return encoded_string
    
    def openai_vision_api(self, image_path: str, prompt: str) -> Dict:
        """Simula chamada para OpenAI Vision API"""
        
        if not self.openai_api_key:
            return {
                "error": "API key not configured",
                "simulation": True,
                "response": {
                    "choices": [{
                        "message": {
                            "content": "Esta é uma simulação da resposta da OpenAI Vision API. A imagem contém um círculo vermelho no centro."
                        }
                    }]
                }
            }
        
        # Codificar imagem
        base64_image = self.encode_image_to_base64(image_path)
        
        # Preparar payload
        payload = {
            "model": "gpt-4-vision-preview",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 300
        }
        
        # Fazer requisição
        headers = {
            "Authorization": f"Bearer {self.openai_api_key}",
            "Content-Type": "application/json"
        }
        
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                json=payload
            )
            
            return response.json()
        
        except Exception as e:
            return {
                "error": str(e),
                "simulation": True,
                "response": {
                    "choices": [{
                        "message": {
                            "content": "Erro na chamada da API. Esta é uma simulação."
                        }
                    }]
                }
            }
    
    def gemini_vision_api(self, image_path: str, prompt: str) -> Dict:
        """Simula chamada para Gemini Vision API"""
        
        if not self.gemini_api_key:
            return {
                "error": "API key not configured",
                "simulation": True,
                "response": {
                    "candidates": [{
                        "content": {
                            "parts": [{
                                "text": "Esta é uma simulação da resposta da Gemini Vision API. A imagem contém um círculo vermelho no centro."
                            }]
                        }
                    }]
                }
            }
        
        # Codificar imagem
        base64_image = self.encode_image_to_base64(image_path)
        
        # Preparar payload
        payload = {
            "contents": [
                {
                    "parts": [
                        {
                            "text": prompt
                        },
                        {
                            "inline_data": {
                                "mime_type": "image/png",
                                "data": base64_image
                            }
                        }
                    ]
                }
            ]
        }
        
        # Fazer requisição
        url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-pro-vision:generateContent?key={self.gemini_api_key}"
        
        try:
            response = requests.post(url, json=payload)
            
            return response.json()
        
        except Exception as e:
            return {
                "error": str(e),
                "simulation": True,
                "response": {
                    "candidates": [{
                        "content": {
                            "parts": [{
                                "text": "Erro na chamada da API. Esta é uma simulação."
                            }]
                        }
                    }]
                }
            }
    
    def compare_apis(self, image_path: str, prompt: str) -> Dict:
        """Compara respostas das APIs"""
        
        print(f"\n=== COMPARAÇÃO DE APIs ===")
        print(f"Prompt: {prompt}")
        print(f"Imagem: {image_path}")
        
        # OpenAI Vision API
        print("\n--- OpenAI Vision API ---")
        openai_response = self.openai_vision_api(image_path, prompt)
        
        if "error" in openai_response:
            print(f"Erro: {openai_response['error']}")
            if openai_response.get("simulation"):
                print("Modo simulação ativado")
        
        if "response" in openai_response:
            openai_text = openai_response["response"]["choices"][0]["message"]["content"]
            print(f"Resposta: {openai_text}")
        
        # Gemini Vision API
        print("\n--- Gemini Vision API ---")
        gemini_response = self.gemini_vision_api(image_path, prompt)
        
        if "error" in gemini_response:
            print(f"Erro: {gemini_response['error']}")
            if gemini_response.get("simulation"):
                print("Modo simulação ativado")
        
        if "response" in gemini_response:
            gemini_text = gemini_response["response"]["candidates"][0]["content"]["parts"][0]["text"]
            print(f"Resposta: {gemini_text}")
        
        return {
            "openai": openai_response,
            "gemini": gemini_response
        }
    
    def analyze_responses(self, responses: Dict) -> Dict:
        """Analisa as respostas das APIs"""
        
        analysis = {
            "openai": {
                "success": "error" not in responses["openai"],
                "simulation": responses["openai"].get("simulation", False),
                "response_length": 0
            },
            "gemini": {
                "success": "error" not in responses["gemini"],
                "simulation": responses["gemini"].get("simulation", False),
                "response_length": 0
            }
        }
        
        # Analisar OpenAI
        if "response" in responses["openai"]:
            openai_text = responses["openai"]["response"]["choices"][0]["message"]["content"]
            analysis["openai"]["response_length"] = len(openai_text)
        
        # Analisar Gemini
        if "response" in responses["gemini"]:
            gemini_text = responses["gemini"]["response"]["candidates"][0]["content"]["parts"][0]["text"]
            analysis["gemini"]["response_length"] = len(gemini_text)
        
        # Visualizar análise
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        # Gráfico de sucesso
        success_data = [analysis["openai"]["success"], analysis["gemini"]["success"]]
        axes[0].bar(["OpenAI", "Gemini"], success_data, color=['blue', 'green'])
        axes[0].set_title('Sucesso das APIs')
        axes[0].set_ylabel('Sucesso')
        axes[0].set_ylim(0, 1)
        
        # Gráfico de comprimento da resposta
        length_data = [analysis["openai"]["response_length"], analysis["gemini"]["response_length"]]
        axes[1].bar(["OpenAI", "Gemini"], length_data, color=['blue', 'green'])
        axes[1].set_title('Comprimento da Resposta')
        axes[1].set_ylabel('Caracteres')
        
        plt.tight_layout()
        plt.show()
        
        return analysis

def demonstrate_apis():
    """Demonstra o uso de APIs para Foundation Models"""
    
    print("=== Demonstração de APIs para Foundation Models ===")
    print("\nNota: Esta demonstração usa modo simulação quando as chaves de API não estão configuradas.")
    print("Para usar as APIs reais, configure as chaves de API.")
    
    # Criar instância
    api_demo = FoundationModelsAPI()
    
    # Criar imagem de exemplo
    print("\nCriando imagem de exemplo...")
    sample_image = api_demo.create_sample_image()
    
    # Testar diferentes prompts
    prompts = [
        "Descreva o que você vê nesta imagem",
        "Quais cores estão presentes na imagem?",
        "Que formas geométricas você identifica?"
    ]
    
    all_responses = []
    
    for prompt in prompts:
        print(f"\n--- Testando prompt: {prompt} ---")
        responses = api_demo.compare_apis('sample_image.png', prompt)
        all_responses.append(responses)
        
        # Analisar respostas
        analysis = api_demo.analyze_responses(responses)
        
        print(f"\nAnálise:")
        print(f"OpenAI - Sucesso: {analysis['openai']['success']}, Simulação: {analysis['openai']['simulation']}")
        print(f"Gemini - Sucesso: {analysis['gemini']['success']}, Simulação: {analysis['gemini']['simulation']}")
    
    return api_demo, all_responses

# Executar demonstração
api_demo, api_responses = demonstrate_apis()

### Análise dos Resultados

**APIs de Foundation Models Observadas:**

1. **OpenAI Vision API**: Análise de imagens com GPT-4V
2. **Gemini Vision API**: Análise de imagens com Gemini
3. **Comparação**: Diferenças entre as APIs
4. **Simulação**: Modo simulação para demonstração

**Insights Importantes:**
- **Multimodalidade**: Processamento de texto e imagem
- **APIs**: Interfaces padronizadas
- **Flexibilidade**: Diferentes modelos disponíveis
- **Integração**: Fácil integração em aplicações

**Referências:**
- [OpenAI API Documentation](https://platform.openai.com/docs)
- [Google Gemini API Documentation](https://ai.google.dev/docs)


## 9.6 Aplicações Práticas e Casos de Uso### Aplicações no Mercado**1. E-commerce:**- **Busca visual**: Busca por produtos usando imagens- **Recomendações**: Recomendações baseadas em visual- **Categorização**: Categorização automática de produtos- **Moderação**: Moderação de conteúdo visual**2. Saúde:**- **Diagnóstico**: Assistência em diagnóstico médico- **Análise**: Análise de imagens médicas- **Documentação**: Documentação automática- **Educação**: Educação médica**3. Educação:**- **Acessibilidade**: Acessibilidade visual- **Tutoria**: Tutoria multimodal- **Conteúdo**: Criação de conteúdo educacional- **Avaliação**: Avaliação automática**4. Entretenimento:**- **Geração**: Geração de conteúdo visual- **Edição**: Edição automática- **Personalização**: Personalização de conteúdo- **Interação**: Interação multimodal![Aplicações Práticas](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo9/aplicacoes_praticas.png)### Vantagens e Limitações**Vantagens:**- **Escalabilidade**: Performance melhora com escala- **Generalização**: Funciona em diferentes domínios- **Flexibilidade**: Adaptável a múltiplas tarefas- **Eficiência**: Reduz necessidade de dados rotulados**Limitações:**- **Recursos**: Requer muitos recursos computacionais- **Dados**: Depende de grandes datasets- **Viés**: Pode perpetuar vieses dos dados- **Interpretabilidade**: Difícil de interpretar### Futuro dos Foundation Models**Tendências:**- **Multimodalidade**: Integração de mais modalidades- **Eficiência**: Modelos mais eficientes- **Especialização**: Modelos especializados- **Acessibilidade**: Maior acessibilidade**Desafios:**- **Ética**: Considerações éticas- **Segurança**: Segurança e privacidade- **Regulamentação**: Regulamentação adequada- **Sustentabilidade**: Impacto ambiental**Referências:**- [On the Opportunities and Risks of Foundation Models - Bommasani et al.](https://arxiv.org/abs/2108.07258)

## Resumo do Módulo 9

### Principais Conceitos Abordados

1. **Foundation Models**
   - Conceitos fundamentais
   - Evolução histórica
   - Características principais

2. **CLIP**
   - Arquitetura e componentes
   - Aprendizado contrastivo
   - Aplicações práticas

3. **DALL-E**
   - Geração de imagens
   - Arquitetura multimodal
   - Aplicações criativas

4. **GPT-4V e Multimodal AI**
   - Processamento multimodal
   - Análise visual
   - Geração de conteúdo

5. **APIs e Aplicações**
   - OpenAI e Gemini APIs
   - Casos de uso práticos
   - Vantagens e limitações

### Demonstrações Práticas

**1. Foundation Models:**
   - Aprendizado contrastivo
   - Classificação zero-shot
   - Visualização de embeddings

**2. APIs:**
   - OpenAI Vision API
   - Gemini Vision API
   - Comparação de respostas

### Próximos Passos

No próximo módulo, exploraremos a **Atividade Final Prática**, onde integraremos todos os conceitos aprendidos em um projeto completo.

### Referências Principais

- [On the Opportunities and Risks of Foundation Models - Bommasani et al.](https://arxiv.org/abs/2108.07258)
- [CLIP: Learning Transferable Visual Representations - Radford et al.](https://arxiv.org/abs/2103.00020)
- [DALL-E: Creating Images from Text - Ramesh et al.](https://arxiv.org/abs/2102.12092)

---

**Próximo Módulo**: Atividade Final Prática
